<a href="https://colab.research.google.com/github/marcocintra/Atmosphere/blob/master/MAGGIA_MAPS_2022_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ZIP FILES ADJUSTS

In [ ]:
# %pwd

In [ ]:
#2022-2024
!curl 'https://my.microsoftpersonalcontent.com/personal/a149071e7d7aaa61/_layouts/15/download.aspx?UniqueId=1e5af62f-acd7-48dd-82c1-7e5a0aa496f6&Translate=false&tempauth=v1e.eyJzaXRlaWQiOiI3NzM0Y2NkNy1mMWE2LTQ0ZmItOWZmYy03MjIyYzI4NGE0ODEiLCJhcHBpZCI6IjAwMDAwMDAwLTAwMDAtMDAwMC0wMDAwLTAwMDA0ODE3MTBhNCIsImF1ZCI6IjAwMDAwMDAzLTAwMDAtMGZmMS1jZTAwLTAwMDAwMDAwMDAwMC9teS5taWNyb3NvZnRwZXJzb25hbGNvbnRlbnQuY29tQDkxODgwNDBkLTZjNjctNGM1Yi1iMTEyLTM2YTMwNGI2NmRhZCIsImV4cCI6IjE3NDUyODQ1MDYifQ.KP2hsGJrZZE5IG-22dt1H-x3cVHJiMgW5I4EN1aDghC9T3XiLHyvbmmmks4mKNDRXJvT6RwgzluaLTEKRE-r97NPbJdqxgfk68P4nO4TG6TXV4nZYrYj3SZuoxt6247uI_T-e5G1Mr7T82DE4qqq209mgy2odi5CPojLhCnMD1M-xyni05GHvFNOO5rsr2Wda1hKPGFnD4ytlEASoOWibjJrpYMI8DXSUi6XkBcy4yqcC4crONuefkhDR4f_s0cxcLpyjhI7he1cVrSJyciMtd0rcsvQB-SZgPiqALMb0oT2CCZ-8he5pcSVbjBnCXoc-7BKTWWnm-atP4JQ7hOkkCqMS7CfUNKmfpIUt-45_1qhpmzDCMgZ6j3Wc4haAB7nquhzKNbKD3U1aUB8kJZKVFetAkRCMjOK30uS8dlYMxI.qLLgx-mxj6bYeJ1wjQBPyqVD2wwU8FQPpr4TGbDdRx8&ApiVersion=2.0&AVOverride=1' \
  -H 'accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7' \
  -H 'accept-language: pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7' \
  -H 'priority: u=0, i' \
  -H 'referer: https://onedrive.live.com/' \
  -H 'sec-ch-ua: "Google Chrome";v="135", "Not-A.Brand";v="8", "Chromium";v="135"' \
  -H 'sec-ch-ua-mobile: ?0' \
  -H 'sec-ch-ua-platform: "Linux"' \
  -H 'sec-fetch-dest: iframe' \
  -H 'sec-fetch-mode: navigate' \
  -H 'sec-fetch-site: cross-site' \
  -H 'sec-fetch-storage-access: none' \
  -H 'sec-fetch-user: ?1' \
  -H 'upgrade-insecure-requests: 1' \
  -H 'user-agent: Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36' > 'MAGGIA-INTERSECTION-2022-2024.zip'

In [ ]:
%pwd

In [ ]:
# !ls

In [ ]:
!unzip -q 'MAGGIA-INTERSECTION-2022-2024.zip'

# DATA FILES ADJUSTS

## 2022

In [ ]:
%cd MAGGIA-INTERSECTION-2022-2024

In [ ]:
%pwd

In [ ]:
%rm *.npy

In [ ]:
!ls

In [ ]:
import os
import glob
from collections import defaultdict

base_path = './'

file_pattern = os.path.join(base_path, '**', 'magn2022*.nc')
file_list = sorted(glob.glob(file_pattern, recursive=True))

file_count_by_day = defaultdict(int)

for file_name in file_list:

    day_dir = os.path.basename(os.path.dirname(file_name))
    file_count_by_day[day_dir] += 1

print("\nDias com menos de 96 arquivos:")

for day, count in sorted(file_count_by_day.items()):
    if count < 96:
        print(f"Dia {day}: {count} arquivos")

In [ ]:
!pip install netCDF4

In [ ]:
!pip install xarray

In [ ]:
import numpy as np
import xarray as xr
import os
import re
import glob

file_list = glob.glob("./magn2022*.nc")

file_list.sort()

tec_by_doy = {}

pattern = re.compile(r'magn2022(\d{3})')

for file_name in file_list:
    try:

        match = pattern.search(file_name)
        if match:
            doy = match.group(1)

            print(f"Processando o arquivo: {file_name}, DOY: {doy}")

            ds_disk = xr.open_dataset(file_name)
            tec = ds_disk['z'][:, :].values

            if doy not in tec_by_doy:
                tec_by_doy[doy] = []

            tec_by_doy[doy].append(tec)

            ds_disk.close()

    except Exception as e:
        print(f"Erro ao processar o arquivo {file_name}: {e}")

print("\nSalvando arquivos NPY por dia...")
for doy, tec_list in tec_by_doy.items():

    tec_array = np.array(tec_list)

    output_file = f"magn2022{doy}.npy"
    np.save(output_file, tec_array)

    print(f"Arquivo {output_file} salvo com sucesso! Shape: {tec_array.shape}")

print("\nProcessamento concluído!")

In [ ]:
# ls *.npy

In [ ]:
!ls -lR ./*.npy | wc -l

In [ ]:
import numpy as np
import glob
import re

npy_files = glob.glob("magn2022*.npy")

def extract_doy(filename):
    match = re.search(r'magn2022(\d{3})\.npy', filename)
    if match:
        return int(match.group(1))
    return 0

npy_files.sort(key=extract_doy)

print(f"Encontrados {len(npy_files)} arquivos NPY para processar.")

all_data = []

for i, file in enumerate(npy_files):
    doy = extract_doy(file)
    print(f"Processando {file} (DOY {doy}) - {i+1}/{len(npy_files)}")

    data = np.load(file)

    all_data.append(data)

    print(f"  Shape: {data.shape}")

combined_data = np.concatenate(all_data, axis=0)

print(f"\nDados combinados. Shape final: {combined_data.shape}")

output_file = "MAGGIA-2022.npy"
np.save(output_file, combined_data)

print(f"\nArquivo {output_file} salvo com sucesso!")
print(f"Tamanho do arquivo: {combined_data.nbytes / (1024**2):.2f} MB")

## 2023

In [ ]:
# %cd MAGGIA-INTERSECTION-2022-2024

In [ ]:
%pwd

In [ ]:
# %rm *.npy

In [ ]:
# !ls

In [ ]:
import os
import glob
from collections import defaultdict

base_path = './'

file_pattern = os.path.join(base_path, '**', 'magn2023*.nc')
file_list = sorted(glob.glob(file_pattern, recursive=True))

file_count_by_day = defaultdict(int)

for file_name in file_list:

    day_dir = os.path.basename(os.path.dirname(file_name))
    file_count_by_day[day_dir] += 1

print("\nDias com menos de 96 arquivos:")

for day, count in sorted(file_count_by_day.items()):
    if count < 96:
        print(f"Dia {day}: {count} arquivos")

In [ ]:
import numpy as np
import xarray as xr
import os
import re
import glob

file_list = glob.glob("./magn2023*.nc")

file_list.sort()

tec_by_doy = {}

pattern = re.compile(r'magn2023(\d{3})')

for file_name in file_list:
    try:

        match = pattern.search(file_name)
        if match:
            doy = match.group(1)

            print(f"Processando o arquivo: {file_name}, DOY: {doy}")

            ds_disk = xr.open_dataset(file_name)
            tec = ds_disk['z'][:, :].values

            if doy not in tec_by_doy:
                tec_by_doy[doy] = []

            tec_by_doy[doy].append(tec)

            ds_disk.close()

    except Exception as e:
        print(f"Erro ao processar o arquivo {file_name}: {e}")

print("\nSalvando arquivos NPY por dia...")
for doy, tec_list in tec_by_doy.items():

    tec_array = np.array(tec_list)

    output_file = f"magn2023{doy}.npy"
    np.save(output_file, tec_array)

    print(f"Arquivo {output_file} salvo com sucesso! Shape: {tec_array.shape}")

print("\nProcessamento concluído!")

In [ ]:
# ls *.npy

In [ ]:
!ls -lR ./*.npy | wc -l

In [ ]:
import numpy as np
import glob
import re

npy_files = glob.glob("magn2023*.npy")

def extract_doy(filename):
    match = re.search(r'magn2023(\d{3})\.npy', filename)
    if match:
        return int(match.group(1))
    return 0

npy_files.sort(key=extract_doy)

print(f"Encontrados {len(npy_files)} arquivos NPY para processar.")

all_data = []

for i, file in enumerate(npy_files):
    doy = extract_doy(file)
    print(f"Processando {file} (DOY {doy}) - {i+1}/{len(npy_files)}")

    data = np.load(file)

    all_data.append(data)

    print(f"  Shape: {data.shape}")

combined_data = np.concatenate(all_data, axis=0)

print(f"\nDados combinados. Shape final: {combined_data.shape}")

output_file = "MAGGIA-2023.npy"
np.save(output_file, combined_data)

print(f"\nArquivo {output_file} salvo com sucesso!")
print(f"Tamanho do arquivo: {combined_data.nbytes / (1024**2):.2f} MB")

## 2024

In [ ]:
# %cd MAGGIA-INTERSECTION-2022-2024

In [ ]:
%pwd

In [ ]:
# %rm *.npy

In [ ]:
# !ls

In [ ]:
import os
import glob
from collections import defaultdict

base_path = './'

file_pattern = os.path.join(base_path, '**', 'magn2024*.nc')
file_list = sorted(glob.glob(file_pattern, recursive=True))

file_count_by_day = defaultdict(int)

for file_name in file_list:

    day_dir = os.path.basename(os.path.dirname(file_name))
    file_count_by_day[day_dir] += 1

print("\nDias com menos de 96 arquivos:")

for day, count in sorted(file_count_by_day.items()):
    if count < 96:
        print(f"Dia {day}: {count} arquivos")

In [ ]:
import numpy as np
import xarray as xr
import os
import re
import glob

file_list = glob.glob("./magn2024*.nc")

file_list.sort()

tec_by_doy = {}

pattern = re.compile(r'magn2024(\d{3})')

for file_name in file_list:
    try:

        match = pattern.search(file_name)
        if match:
            doy = match.group(1)

            print(f"Processando o arquivo: {file_name}, DOY: {doy}")

            ds_disk = xr.open_dataset(file_name)
            tec = ds_disk['z'][:, :].values

            if doy not in tec_by_doy:
                tec_by_doy[doy] = []

            tec_by_doy[doy].append(tec)

            ds_disk.close()

    except Exception as e:
        print(f"Erro ao processar o arquivo {file_name}: {e}")

print("\nSalvando arquivos NPY por dia...")
for doy, tec_list in tec_by_doy.items():

    tec_array = np.array(tec_list)

    output_file = f"magn2024{doy}.npy"
    np.save(output_file, tec_array)

    print(f"Arquivo {output_file} salvo com sucesso! Shape: {tec_array.shape}")

print("\nProcessamento concluído!")

In [ ]:
# ls *.npy

In [ ]:
# !ls -lR ./*.npy | wc -l

In [ ]:
import numpy as np
import glob
import re

npy_files = glob.glob("magn2024*.npy")

def extract_doy(filename):
    match = re.search(r'magn2024(\d{3})\.npy', filename)
    if match:
        return int(match.group(1))
    return 0

npy_files.sort(key=extract_doy)

print(f"Encontrados {len(npy_files)} arquivos NPY para processar.")

all_data = []

for i, file in enumerate(npy_files):
    doy = extract_doy(file)
    print(f"Processando {file} (DOY {doy}) - {i+1}/{len(npy_files)}")

    data = np.load(file)

    all_data.append(data)

    print(f"  Shape: {data.shape}")

combined_data = np.concatenate(all_data, axis=0)

print(f"\nDados combinados. Shape final: {combined_data.shape}")

output_file = "MAGGIA-2024.npy"
np.save(output_file, combined_data)

print(f"\nArquivo {output_file} salvo com sucesso!")
print(f"Tamanho do arquivo: {combined_data.nbytes / (1024**2):.2f} MB")

# 5D ARRAY - MERGE 2022-2024 DATA

In [ ]:
%cd './'

In [ ]:
import numpy as np

In [ ]:
# %cd /content/MAGGIA-INTERSECTION-2022-2024

In [ ]:
# !ls *.npy

In [ ]:
%pwd

In [ ]:
!ls

In [ ]:
%pwd

In [ ]:
!ls *.npy

In [ ]:
maggia_2022 = np.load('MAGGIA-2022.npy')
print(np.shape(maggia_2022))
print(type(maggia_2022))
print(type(maggia_2022[0]))
print(type(maggia_2022[0][0]))
print(type(maggia_2022[0][0][0]))

In [ ]:
for i in range(len(maggia_2022)):
    print(np.shape(maggia_2022[i]))

In [ ]:
np.shape(maggia_2022)

In [ ]:
has_nan = np.any(np.isnan(maggia_2022))
print(has_nan)

count_nan = np.sum(np.isnan(maggia_2022))
print(count_nan)

In [ ]:
maggia_2022.shape

In [ ]:
np.set_printoptions(threshold=np.inf)

In [ ]:
print(maggia_2022[0][0])

In [ ]:
print(maggia_2022[0][-1])

In [ ]:
# del maggia_2022

In [ ]:
maggia_2023 = np.load('MAGGIA-2023.npy')
print(np.shape(maggia_2023))
print(type(maggia_2023))
print(type(maggia_2023[0]))
print(type(maggia_2023[0][0]))
print(type(maggia_2023[0][0][0]))

In [ ]:
for i in range(len(maggia_2023)):
    print(np.shape(maggia_2023[i]))

In [ ]:
np.shape(maggia_2023)

In [ ]:
has_nan = np.any(np.isnan(maggia_2023))
print(has_nan)

count_nan = np.sum(np.isnan(maggia_2023))
print(count_nan)

In [ ]:
maggia_2023.shape

In [ ]:
np.set_printoptions(threshold=np.inf)

In [ ]:
print(maggia_2023[0][0])

In [ ]:
print(maggia_2023[0][-1])

In [ ]:
# del maggia_2023

In [ ]:
maggia_2024 = np.load('MAGGIA-2024.npy')
print(np.shape(maggia_2024))
print(type(maggia_2024))
print(type(maggia_2024[0]))
print(type(maggia_2024[0][0]))
print(type(maggia_2024[0][0][0]))

In [ ]:
for i in range(len(maggia_2024)):
    print(np.shape(maggia_2024[i]))

In [ ]:
np.shape(maggia_2024)

In [ ]:
has_nan = np.any(np.isnan(maggia_2024))
print(has_nan)

count_nan = np.sum(np.isnan(maggia_2024))
print(count_nan)

In [ ]:
maggia_2024.shape

In [ ]:
np.set_printoptions(threshold=np.inf)

In [ ]:
print(maggia_2024[0][0])

In [ ]:
print(maggia_2024[0][-1])

In [ ]:
# del maggia_2024

In [ ]:
maggia_2022_2024 = np.concatenate([maggia_2022, maggia_2023, maggia_2024], axis=0)
print(maggia_2022_2024.shape)

In [ ]:
print(type(maggia_2022_2024))
print(type(maggia_2022_2024[0]))
print(type(maggia_2022_2024[0][0]))
print(type(maggia_2022_2024[0][0][0]))
# print(type(maggia_2022_2024[0][0][0][0]))

In [ ]:
for i in range(len(maggia_2022_2024)):
    print(np.shape(maggia_2022_2024[i]))

In [ ]:
np.shape(maggia_2022_2024)

In [ ]:
has_nan = np.any(np.isnan(maggia_2022_2024))
print(has_nan)

count_nan = np.sum(np.isnan(maggia_2022_2024))
print(count_nan)

In [ ]:
np.save('maggia_2022_2024.npy', maggia_2022_2024)

In [ ]:
np.set_printoptions(threshold=np.inf)

In [ ]:
print(maggia_2022_2024[0][0])

In [ ]:
print(maggia_2022_2024[0][-1])

# DATES ADJUSTS

In [ ]:
import numpy as np
from datetime import datetime, timedelta

def doy_to_date(year, doy):

    return datetime(year, 1, 1) + timedelta(days=doy - 1)

doys = {
    2022: {
        3: [60, 61, 62, 63, 64, 65, 66, 67, 69, 70,
            71, 73, 74, 75, 76, 77, 78, 79, 80, 81,
            82, 83, 84, 85, 86, 90],
        6: [152, 153, 154, 156, 157, 158, 159, 160,
            161, 162, 163, 164, 165, 166, 167, 168,
            169, 170, 171, 172, 173, 174, 176, 178,
            181],
        9: [245, 246, 247, 248, 249, 250, 251, 252,
            253, 254, 255, 256, 258, 260, 261, 262,
            264, 265, 266, 267, 268, 269, 270, 271,
            272, 273],
        12: [338, 339, 340, 348, 349, 350, 351, 352,
             353, 354, 356, 357, 358, 359, 360, 362,
             363, 364, 365]
    },
    2023: {
        3: [61, 62, 63, 64, 65, 66, 67, 68, 69, 70,
            71, 72, 74, 75, 76, 77, 78, 79, 80, 81,
            82, 83, 84, 85, 86, 87, 88, 89, 90],
        6: [157, 158, 159, 160, 161, 162, 163, 164, 165, 166,
            167, 168, 169, 170, 171, 172, 173, 174, 175, 176,
            177, 178, 179, 180, 181],
        9: [244, 245, 246, 247, 249, 250, 262, 263, 264, 265,
            266, 267, 268, 269, 270, 271, 272, 273],
        12: [335, 337, 338, 339, 341, 342, 343, 344, 345,
             346, 347, 348, 349, 350, 351, 352, 353, 355, 356,
             357, 358, 359, 360, 361, 362, 363, 365]
    },
    2024: {
        3: [61, 62, 63, 64, 65, 66, 67, 68, 69, 70,
            71, 72, 73, 74, 75, 76, 77, 78, 79, 80,
            81, 86, 87, 88, 89, 91],
        6: [153, 154, 155, 156, 157, 158, 159, 160, 161, 162,
            163, 164, 165, 166, 167, 168, 169, 170, 171, 173,
            174, 175, 176, 177, 180, 181],
        9: [247, 248, 249, 250, 251, 252, 253, 254, 255, 256,
            257, 258, 261, 262, 263, 264, 265, 266, 267, 268,
            269, 270, 271, 272, 274],
        12: [336, 337, 338, 339, 340, 341, 342, 343, 344, 345,
             346, 347, 349, 350, 351, 352, 353, 354, 355,
             356, 357, 358, 359, 360, 361, 362, 363, 364, 365]
    }
}

doy249_times = [
    "0000", "0010", "0015", "0020", "0030", "0040", "0045", "0050",
    "0100", "0110", "0115", "0120", "0130", "0140", "0145", "0150",
    "0200", "0210", "0215", "0220", "0230", "0240", "0245", "0250",
    "0300", "0310", "0315", "0320", "0330", "0340", "0345", "0350",
    "0400", "0410", "0415", "0420", "0430", "0440", "0445", "0450",
    "0500", "0510", "0515", "0520", "0530", "0540", "0545", "0550",
    "0600", "0610", "0615", "0620", "0630", "0640", "0645", "0650",
    "0700", "0710", "0715", "0720", "0730", "0740", "0745", "0750",
    "0800", "0810", "0815", "0820", "0830", "0840", "0845", "0850",
    "0900", "0910", "0915", "0920", "0930", "0940", "0945", "0950",
    "1000", "1010", "1015", "1020", "1030", "1040", "1045", "1050",
    "1100", "1110", "1115", "1120", "1130", "1140", "1145", "1150",
    "1200", "1210", "1215", "1220", "1230", "1240", "1245", "1250",
    "1300", "1310", "1315", "1320", "1330", "1340", "1345", "1350",
    "1400", "1410", "1415", "1420", "1430", "1440", "1445", "1450",
    "1500", "1510", "1515", "1520", "1530", "1540", "1545", "1550",
    "1600", "1610", "1615", "1620", "1630", "1640", "1645", "1650",
    "1700", "1710", "1715", "1720", "1730", "1740", "1745", "1750",
    "1800", "1810", "1815", "1820", "1830", "1840", "1845", "1850",
    "1900", "1910", "1915", "1920", "1930", "1940", "1945", "1950",
    "2000", "2010", "2015", "2020", "2030", "2040", "2045", "2050",
    "2100", "2110", "2115", "2120", "2130", "2140", "2145", "2150",
    "2200", "2210", "2215", "2220", "2230", "2240", "2245", "2250",
    "2300", "2310", "2315", "2320", "2330", "2340", "2345", "2350"
]

doy250_times = [
    "0000", "0010", "0015", "0020", "0030", "0040", "0045", "0050",
    "0100", "0110", "0115", "0120", "0130", "0140", "0145", "0150",
    "0200", "0210", "0215", "0220", "0230", "0240", "0245", "0250",
    "0300", "0310", "0315", "0320", "0330", "0340", "0345", "0350",
    "0400", "0410", "0415", "0420", "0430", "0440", "0445", "0450",
    "0500", "0510", "0515", "0520", "0530", "0540", "0545", "0550",
    "0600", "0610", "0615", "0620", "0630", "0640", "0645", "0650",
    "0700", "0710", "0715", "0720", "0730", "0740", "0745", "0750",
    "0800", "0810", "0815", "0820", "0830", "0840", "0845", "0850",
    "0900", "0910", "0915", "0920", "0930", "0940", "0945", "0950",
    "1000", "1010", "1015", "1020", "1030", "1040", "1045", "1050",
    "1100", "1110", "1115", "1120", "1130", "1140", "1145", "1150",
    "1200", "1210", "1220", "1230", "1240", "1250", "1300", "1310",
    "1320", "1330", "1340", "1350", "1400", "1410", "1420", "1430",
    "1440", "1450", "1500", "1510", "1520", "1530", "1540", "1550",
    "1600", "1610", "1620", "1630", "1640", "1650", "1700", "1710",
    "1720", "1730", "1740", "1750", "1800", "1810", "1820", "1830",
    "1840", "1850", "1900", "1910", "1920", "1930", "1940", "1950",
    "2000", "2010", "2020", "2030", "2040", "2050", "2100", "2110",
    "2120", "2130", "2140", "2150", "2200", "2210", "2220", "2230",
    "2240", "2250", "2300", "2310", "2320", "2330", "2340", "2350"
]

datetime_list = []

for year in doys.keys():
    for month in doys[year].keys():
        for doy in doys[year][month]:
            base_date = doy_to_date(year, doy)

            if year == 2024 and doy == 249:
                for time_str in doy249_times:
                    hours = int(time_str[:2])
                    minutes = int(time_str[2:])
                    dt = base_date + timedelta(hours=hours, minutes=minutes)
                    datetime_list.append(dt)

            elif year == 2024 and doy == 250:
                for time_str in doy250_times:
                    hours = int(time_str[:2])
                    minutes = int(time_str[2:])
                    dt = base_date + timedelta(hours=hours, minutes=minutes)
                    datetime_list.append(dt)

            elif year == 2024 and doy >= 251:
                for minute in range(0, 24 * 60, 10):
                    dt = base_date + timedelta(minutes=minute)
                    datetime_list.append(dt)

            else:
                for minute in range(0, 24 * 60, 15):
                    dt = base_date + timedelta(minutes=minute)
                    datetime_list.append(dt)

maggia_datetime_2022_2024 = np.array(datetime_list, dtype='datetime64[s]')

print(f"Total number of datetime points: {len(maggia_datetime_2022_2024)}")
print(f"First datetime: {maggia_datetime_2022_2024[0]}")
print(f"Last datetime: {maggia_datetime_2022_2024[-1]}")

np.save('maggia_datetime_2022_2024.npy', maggia_datetime_2022_2024)

print("\nSample of first 10 datetime entries:")
for dt in maggia_datetime_2022_2024[:10]:
    print(dt)

print("\nSample of last 10 datetime entries:")
for dt in maggia_datetime_2022_2024[-10:]:
    print(dt)

In [ ]:
maggia_dates = np.load('maggia_datetime_2022_2024.npy')

In [ ]:
maggia_dates.shape

In [ ]:
maggia_dates

In [ ]:
maggia_2022_2024.shape

In [ ]:
print(len(maggia_2022_2024.tolist()))
print(len(maggia_2022_2024[0].tolist()))
print(len(maggia_2022_2024[0][0].tolist()))
# print(len(maggia_2022_2024[0][0][0].tolist()))

In [ ]:
maggia_2022.shape

In [ ]:
maggia_2022.shape

In [ ]:
maggia_2023.shape

In [ ]:
maggia_2024.shape

In [ ]:
maggia_dates_2022 = np.array([d for d in maggia_dates if d.astype(object).year == 2022])

In [ ]:
len(maggia_dates_2022)

In [ ]:
maggia_dates_2023 = np.array([d for d in maggia_dates if d.astype(object).year == 2023])

In [ ]:
len(maggia_dates_2023)

In [ ]:
maggia_dates_2024 = np.array([d for d in maggia_dates if d.astype(object).year == 2024])

In [ ]:
len(maggia_dates_2024)

In [ ]:
maggia_dates[0:12]

In [ ]:
len(maggia_dates[0:12])

In [ ]:
len(maggia_2022_2024[0:12])

In [ ]:
maggia_2022_2024_shaped = np.reshape(maggia_2022_2024, (-1, 241, 221))

In [ ]:
np.shape(maggia_2022_2024_shaped)

In [ ]:
np.shape(maggia_2022_2024_shaped.tolist())

In [ ]:
maggia_dates.shape

## CREATE DATAFRAME

In [ ]:
import pandas as pd

data = {
    'DATETIME': maggia_dates,
    'TECMAP': maggia_2022_2024_shaped.tolist()
}

df_mapas_maggia_2022_2024 = pd.DataFrame(data)

df_mapas_maggia_2022_2024

In [ ]:
df_mapas_maggia_2022_2024['DATETIME'] = pd.to_datetime(df_mapas_maggia_2022_2024['DATETIME'])

df_mapas_maggia_2022_2024.set_index('DATETIME', inplace=True)

df_mapas_maggia_2022_2024 = df_mapas_maggia_2022_2024[(df_mapas_maggia_2022_2024.index.minute == 0) & (df_mapas_maggia_2022_2024.index.hour % 2 == 0)]

df1 = df_mapas_maggia_2022_2024[
    ((df_mapas_maggia_2022_2024.index.year == 2022) & (df_mapas_maggia_2022_2024.index.month.isin([3, 6, 9]))) |
    ((df_mapas_maggia_2022_2024.index.year == 2024) & (df_mapas_maggia_2022_2024.index.month.isin([3, 6, 9, 12])))
]

df2 = df_mapas_maggia_2022_2024[
    ((df_mapas_maggia_2022_2024.index.year == 2022) & (df_mapas_maggia_2022_2024.index.month == 12)) |
    ((df_mapas_maggia_2022_2024.index.year == 2023) & (df_mapas_maggia_2022_2024.index.month.isin([3, 6, 9, 12])))
]

print("DataFrame 1:")
print(df1)
print("\nDataFrame 2:")
print(df2)

In [ ]:
np.array(df_mapas_maggia_2022_2024.iloc[0]['TECMAP'])

In [ ]:
np.array(df_mapas_maggia_2022_2024.iloc[0]['TECMAP']).shape

In [ ]:
df_mapas_maggia_2022_2024.to_pickle("./df_mapas_maggia_2022_2024.pkl")

In [ ]:
mapas = np.array(df_mapas_maggia_2022_2024.iloc[:]['TECMAP'])

In [ ]:
np.shape(df_mapas_maggia_2022_2024)

In [ ]:
np_mapas = []
for i in range(len(mapas)):
    np_mapas.append(mapas[i])
np_mapas = np.array(np_mapas)

In [ ]:
np.shape(np_mapas)

In [ ]:
type(np_mapas)

In [ ]:
np.save('mapas_maggia_2022_2024.npy', np_mapas)

In [ ]:
df_mapas_maggia_2022_2024_0800 = df_mapas_maggia_2022_2024.between_time('08:00', '08:00')

df_mapas_maggia_2022_2024_1600 = df_mapas_maggia_2022_2024.between_time('16:00', '16:00')

df_mapas_maggia_2022_2024_2000_2200_0000_0200_0400 = pd.concat([
    df_mapas_maggia_2022_2024.between_time('20:00', '20:00'),
    df_mapas_maggia_2022_2024.between_time('22:00', '22:00'),
    df_mapas_maggia_2022_2024.between_time('00:00', '00:00'),
    df_mapas_maggia_2022_2024.between_time('02:00', '02:00'),
    df_mapas_maggia_2022_2024.between_time('04:00', '04:00')
])

print("Dados das 08:00:")
print(df_mapas_maggia_2022_2024_0800)

print("\nDados das 16:00:")
print(df_mapas_maggia_2022_2024_1600)

print("\nDados das 20:00, 22:00, 00:00, 02:00 e 04:00:")
print(df_mapas_maggia_2022_2024_2000_2200_0000_0200_0400)

In [ ]:
df_mapas_maggia_2022_2024_0800.to_pickle("./df_mapas_maggia_2022_2024_0800.pkl")

In [ ]:
np.shape(np.array(df_mapas_maggia_2022_2024_0800.iloc[:]['TECMAP']))

In [ ]:
mapas_maggia_2022_2024_0800 = np.array(df_mapas_maggia_2022_2024_0800.iloc[:]['TECMAP'])

In [ ]:
np.shape(mapas_maggia_2022_2024_0800)

In [ ]:
np_mapas_maggia_2022_2024_0800 = []
for i in range(len(mapas_maggia_2022_2024_0800)):
    np_mapas_maggia_2022_2024_0800.append(mapas_maggia_2022_2024_0800[i])
np_mapas_maggia_2022_2024_0800 = np.array(np_mapas_maggia_2022_2024_0800)

In [ ]:
np.shape(np_mapas_maggia_2022_2024_0800)

In [ ]:
type(np_mapas_maggia_2022_2024_0800)

In [ ]:
np.save('mapas_maggia_2022_2024_0800.npy', np_mapas_maggia_2022_2024_0800)

In [ ]:
df_mapas_maggia_2022_2024_1600.to_pickle("./df_mapas_maggia_2022_2024_1600.pkl")

In [ ]:
mapas_maggia_2022_2024_1600 = np.array(df_mapas_maggia_2022_2024_1600.iloc[:]['TECMAP'])

In [ ]:
np.shape(mapas_maggia_2022_2024_1600)

In [ ]:
np_mapas_maggia_2022_2024_1600 = []
for i in range(len(mapas_maggia_2022_2024_1600)):
    np_mapas_maggia_2022_2024_1600.append(mapas_maggia_2022_2024_1600[i])
np_mapas_maggia_2022_2024_1600 = np.array(np_mapas_maggia_2022_2024_1600)

In [ ]:
np.shape(np_mapas_maggia_2022_2024_1600)

In [ ]:
type(np_mapas_maggia_2022_2024_1600)

In [ ]:
np.save('mapas_maggia_2022_2024_1600.npy', np_mapas_maggia_2022_2024_1600)

In [ ]:
df_mapas_maggia_2022_2024_2000_2200_0000_0200_0400.to_pickle("./df_mapas_maggia_2022_2024_2000_2200_0000_0200_0400.pkl")

In [ ]:
mapas_maggia_2022_2024_2000_2200_0000_0200_0400 = np.array(df_mapas_maggia_2022_2024_2000_2200_0000_0200_0400.iloc[:]['TECMAP'])

In [ ]:
np.shape(mapas_maggia_2022_2024_2000_2200_0000_0200_0400)

In [ ]:
np.shape(mapas_maggia_2022_2024_2000_2200_0000_0200_0400[0])

In [ ]:
np_mapas_maggia_2022_2024_2000_2200_0000_0200_0400 = []
for i in range(len(mapas_maggia_2022_2024_2000_2200_0000_0200_0400)):
    np_mapas_maggia_2022_2024_2000_2200_0000_0200_0400.append(mapas_maggia_2022_2024_2000_2200_0000_0200_0400[i])
np_mapas_maggia_2022_2024_2000_2200_0000_0200_0400 = np.array(np_mapas_maggia_2022_2024_2000_2200_0000_0200_0400)

In [ ]:
np.shape(np_mapas_maggia_2022_2024_2000_2200_0000_0200_0400)

In [ ]:
type(np_mapas_maggia_2022_2024_2000_2200_0000_0200_0400)

In [ ]:
np.save('mapas_maggia_2022_2024_2000_2200_0000_0200_0400.npy', np_mapas_maggia_2022_2024_2000_2200_0000_0200_0400)

# TF1 ADJUSTS

## CONTINUOUS

In [ ]:
np.array(df1.iloc[0]['TECMAP'])

In [ ]:
np.array(df1.iloc[0]['TECMAP']).shape

In [ ]:
df1_0800 = df1.between_time('08:00', '08:00')

df1_1600 = df1.between_time('16:00', '16:00')

df1_2000_2200_0000_0200_0400 = pd.concat([
    df1.between_time('20:00', '20:00'),
    df1.between_time('22:00', '22:00'),
    df1.between_time('00:00', '00:00'),
    df1.between_time('02:00', '02:00'),
    df1.between_time('04:00', '04:00')
])

df1_2000_2200_0000_0200_0400.sort_index(inplace=True)

print("Dados das 08:00:")
print(df1_0800)

print("\nDados das 16:00:")
print(df1_1600)

print("\nDados das 20:00, 22:00, 00:00, 02:00 e 04:00:")
print(df1_2000_2200_0000_0200_0400)

In [ ]:
df1.to_pickle("./df1_mapas_maggia_2022_2024.pkl")

In [ ]:
np.shape(np.array(df1.iloc[:]['TECMAP']))

In [ ]:
mapas1 = np.array(df1.iloc[:]['TECMAP'])

In [ ]:
np.shape(df1)

In [ ]:
np_mapas1 = []
for i in range(len(mapas1)):
    np_mapas1.append(mapas1[i])
np_mapas1 = np.array(np_mapas1)

In [ ]:
np.shape(np_mapas1)

In [ ]:
type(np_mapas1)

In [ ]:
np.save('mapas1_maggia_2022_2024.npy', np_mapas1)

## 08:00 UTC

In [ ]:
df1_0800.to_pickle("./df1_mapas_maggia_2022_2024_0800.pkl")

In [ ]:
np.shape(np.array(df1_0800.iloc[:]['TECMAP']))

In [ ]:
mapas1_0800 = np.array(df1_0800.iloc[:]['TECMAP'])

In [ ]:
np.shape(df1_0800)

In [ ]:
np_mapas1_0800 = []
for i in range(len(mapas1_0800)):
    np_mapas1_0800.append(mapas1_0800[i])
np_mapas1_0800 = np.array(np_mapas1_0800)

In [ ]:
np.shape(np_mapas1_0800)

In [ ]:
type(np_mapas1_0800)

In [ ]:
np.save('mapas1_maggia_2022_2024_0800.npy', np_mapas1_0800)

## 16:00 UTC

In [ ]:
df1_1600.to_pickle("./df1_mapas_maggia_2022_2024_1600.pkl")

In [ ]:
mapas1_1600 = np.array(df1_1600.iloc[:]['TECMAP'])

In [ ]:
np.shape(mapas1_1600)

In [ ]:
np_mapas1_1600 = []
for i in range(len(mapas1_1600)):
    np_mapas1_1600.append(mapas1_1600[i])
np_mapas1_1600 = np.array(np_mapas1_1600)

In [ ]:
np.shape(np_mapas1_1600)

In [ ]:
type(np_mapas1_1600)

In [ ]:
np.save('mapas1_maggia_2022_2024_1600.npy', np_mapas1_1600)

## 20:00 TO 04:00 UTC

In [ ]:
df1_2000_2200_0000_0200_0400.to_pickle("./df1_mapas_maggia_2022_2024_2000_2200_0000_0200_0400.pkl")

In [ ]:
mapas1_2000_2200_0000_0200_0400 = np.array(df1_2000_2200_0000_0200_0400.iloc[:]['TECMAP'])

In [ ]:
np.shape(mapas1_2000_2200_0000_0200_0400)

In [ ]:
np_mapas1_2000_2200_0000_0200_0400 = []
for i in range(len(mapas1_2000_2200_0000_0200_0400)):
    np_mapas1_2000_2200_0000_0200_0400.append(mapas1_2000_2200_0000_0200_0400[i])
np_mapas1_2000_2200_0000_0200_0400 = np.array(np_mapas1_2000_2200_0000_0200_0400)

In [ ]:
np.shape(np_mapas1_2000_2200_0000_0200_0400)

In [ ]:
type(np_mapas1_2000_2200_0000_0200_0400)

In [ ]:
np.save('mapas1_maggia_2022_2024_2000_2200_0000_0200_0400.npy', np_mapas1_2000_2200_0000_0200_0400)

# TF2 ADJUSTS

## CONTINUOUS

In [ ]:
np.array(df2.iloc[0]['TECMAP'])

In [ ]:
np.array(df2.iloc[0]['TECMAP']).shape

In [ ]:
df2_0800 = df2.between_time('08:00', '08:00')

df2_1600 = df2.between_time('16:00', '16:00')

df2_2000_2200_0000_0200_0400 = pd.concat([
    df2.between_time('20:00', '20:00'),
    df2.between_time('22:00', '22:00'),
    df2.between_time('00:00', '00:00'),
    df2.between_time('02:00', '02:00'),
    df2.between_time('04:00', '04:00')
])

df2_2000_2200_0000_0200_0400.sort_index(inplace=True)

print("Dados das 08:00:")
print(df2_0800)

print("\nDados das 16:00:")
print(df2_1600)

print("\nDados das 20:00, 22:00, 00:00, 02:00 e 04:00:")
print(df2_2000_2200_0000_0200_0400)

In [ ]:
df2.to_pickle("./df2_mapas_maggia_2022_2024.pkl")

In [ ]:
np.shape(np.array(df2.iloc[:]['TECMAP']))

In [ ]:
mapas2 = np.array(df2.iloc[:]['TECMAP'])

In [ ]:
np.shape(df2)

In [ ]:
np_mapas2 = []
for i in range(len(mapas2)):
    np_mapas2.append(mapas2[i])
np_mapas2 = np.array(np_mapas2)

In [ ]:
np.shape(np_mapas2)

In [ ]:
type(np_mapas2)

In [ ]:
np.save('mapas2_maggia_2022_2024.npy', np_mapas2)

## 08:00 UTC

In [ ]:
df2_0800.to_pickle("./df2_mapas_maggia_2022_2024_0800.pkl")

In [ ]:
np.shape(np.array(df2_0800.iloc[:]['TECMAP']))

In [ ]:
mapas2_0800 = np.array(df2_0800.iloc[:]['TECMAP'])

In [ ]:
np.shape(df2_0800)

In [ ]:
np_mapas2_0800 = []
for i in range(len(mapas2_0800)):
    np_mapas2_0800.append(mapas2_0800[i])
np_mapas2_0800 = np.array(np_mapas2_0800)

In [ ]:
np.shape(np_mapas2_0800)

In [ ]:
type(np_mapas2_0800)

In [ ]:
np.save('mapas2_maggia_2022_2024_0800.npy', np_mapas2_0800)

## 16:00 UTC

In [ ]:
df2_1600.to_pickle("./df2_mapas_maggia_2022_2024_1600.pkl")

In [ ]:
mapas2_1600 = np.array(df2_1600.iloc[:]['TECMAP'])

In [ ]:
np.shape(mapas2_1600)

In [ ]:
np_mapas2_1600 = []
for i in range(len(mapas2_1600)):
    np_mapas2_1600.append(mapas2_1600[i])
np_mapas2_1600 = np.array(np_mapas2_1600)

In [ ]:
np.shape(np_mapas2_1600)

In [ ]:
type(np_mapas2_1600)

In [ ]:
np.save('mapas2_maggia_2022_2024_1600.npy', np_mapas2_1600)

## 20:00 TO 04:00 UTC

In [ ]:
df2_2000_2200_0000_0200_0400.to_pickle("./df2_mapas_maggia_2022_2024_2000_2200_0000_0200_0400.pkl")

In [ ]:
mapas2_2000_2200_0000_0200_0400 = np.array(df2_2000_2200_0000_0200_0400.iloc[:]['TECMAP'])

In [ ]:
np.shape(mapas2_2000_2200_0000_0200_0400)

In [ ]:
np_mapas2_2000_2200_0000_0200_0400 = []
for i in range(len(mapas2_2000_2200_0000_0200_0400)):
    np_mapas2_2000_2200_0000_0200_0400.append(mapas2_2000_2200_0000_0200_0400[i])
np_mapas2_2000_2200_0000_0200_0400 = np.array(np_mapas2_2000_2200_0000_0200_0400)

In [ ]:
np.shape(np_mapas2_2000_2200_0000_0200_0400)

In [ ]:
type(np_mapas2_2000_2200_0000_0200_0400)

In [ ]:
np.save('mapas2_maggia_2022_2024_2000_2200_0000_0200_0400.npy', np_mapas2_2000_2200_0000_0200_0400)

# CASE STUDY - 30-MINUTE TEMPORAL RESOLUTION - INTERSECTION BETWEEN NAGOYA, MAGGIA AND EMBRACE

# ZIP FILES ADJUSTS

In [ ]:
# %pwd

In [ ]:
!curl 'https://my.microsoftpersonalcontent.com/personal/a149071e7d7aaa61/_layouts/15/download.aspx?UniqueId=1e5af62f-acd7-48dd-82c1-7e5a0aa496f6&Translate=false&tempauth=v1e.eyJzaXRlaWQiOiI3NzM0Y2NkNy1mMWE2LTQ0ZmItOWZmYy03MjIyYzI4NGE0ODEiLCJhcHBpZCI6IjAwMDAwMDAwLTAwMDAtMDAwMC0wMDAwLTAwMDA0ODE3MTBhNCIsImF1ZCI6IjAwMDAwMDAzLTAwMDAtMGZmMS1jZTAwLTAwMDAwMDAwMDAwMC9teS5taWNyb3NvZnRwZXJzb25hbGNvbnRlbnQuY29tQDkxODgwNDBkLTZjNjctNGM1Yi1iMTEyLTM2YTMwNGI2NmRhZCIsImV4cCI6IjE3NDQ1Nzg2NTUifQ.1pGkQfLVNOVounEsVmd_Os7TfrN01eFnqMN9rAgHylNAh_DgzMzRWCGPB45F5mCCtwot5EDzj37ha_5WnBTXSVBStcDmvC5BkiOkA6KdqqhpcNGNitxq57LOseyqj4NxbW8jLe34iPU_w2pCPTA7jvppd0LNUoACB6vUafUBUXQliuyanSBCkcWHEH1qjDyEWpitSJuCld_NE-12dlVwIl9daMStxsiefjlzvHGRFzdVDLFVJ_LpbUrICkIYQD8FzCtHbvcW9ZhrEr7L_OsDUqZdf94R_hBitPY6DL-9CLLBNLGUpWHwME3nrmlzWAfL6Sdl5oL1OUTbQ4buVLicPcVo3dMYSLx7Icybwd-UsZcqVdRS1HusYhCTLZgySEfV2kKxia3P-5ECcdxcxLjfzUtZsmAbvK3gfXrzB1fdkgY.RCCvw9n0wZNHBHZ2UWeV1bJr606XqaDve7wi4U9sW9Y&ApiVersion=2.0&AVOverride=1' \
  -H 'accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7' \
  -H 'accept-language: pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7' \
  -H 'priority: u=0, i' \
  -H 'referer: https://onedrive.live.com/' \
  -H 'sec-ch-ua: "Google Chrome";v="135", "Not-A.Brand";v="8", "Chromium";v="135"' \
  -H 'sec-ch-ua-mobile: ?0' \
  -H 'sec-ch-ua-platform: "Linux"' \
  -H 'sec-fetch-dest: iframe' \
  -H 'sec-fetch-mode: navigate' \
  -H 'sec-fetch-site: cross-site' \
  -H 'sec-fetch-storage-access: none' \
  -H 'sec-fetch-user: ?1' \
  -H 'upgrade-insecure-requests: 1' \
  -H 'user-agent: Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36' > 'MAGGIA-INTERSECTION-2022-2024.zip'

In [ ]:
%pwd

In [ ]:
!ls

In [ ]:
!unzip -q 'MAGGIA-INTERSECTION-2022-2024.zip'

# DATA FILES ADJUSTS

## 2024

In [ ]:
!pip install netCDF4

In [ ]:
!pip install xarray

In [ ]:
%cd '/content/MAGGIA-INTERSECTION-2022-2024'

In [ ]:
!ls

In [ ]:
import os
import glob
from collections import defaultdict

base_path = './'

file_pattern = os.path.join(base_path, '**', 'magn2024*.nc')
file_list = sorted(glob.glob(file_pattern, recursive=True))

file_count_by_day = defaultdict(int)

for file_name in file_list:

    day_dir = os.path.basename(os.path.dirname(file_name))
    file_count_by_day[day_dir] += 1

# for day, count in sorted(file_count_by_day.items()):
#     print(f"Dia {day}: {count} arquivos")

print("\nDias com menos de 96 arquivos:")

for day, count in sorted(file_count_by_day.items()):
    if count < 96:
        print(f"Dia {day}: {count} arquivos")

In [ ]:
import numpy as np
import xarray as xr
import os
import re
import glob

file_list = glob.glob("./magn2024*.nc")

file_list.sort()

tec_by_doy = {}

pattern = re.compile(r'magn2024(\d{3})')

for file_name in file_list:
    try:

        match = pattern.search(file_name)
        if match:
            doy = match.group(1)

            print(f"Processando o arquivo: {file_name}, DOY: {doy}")

            ds_disk = xr.open_dataset(file_name)
            tec = ds_disk['z'][:, :].values

            if doy not in tec_by_doy:
                tec_by_doy[doy] = []

            tec_by_doy[doy].append(tec)

            ds_disk.close()

    except Exception as e:
        print(f"Erro ao processar o arquivo {file_name}: {e}")

print("\nSalvando arquivos NPY por dia...")
for doy, tec_list in tec_by_doy.items():

    tec_array = np.array(tec_list)

    output_file = f"magn2024{doy}.npy"
    np.save(output_file, tec_array)

    print(f"Arquivo {output_file} salvo com sucesso! Shape: {tec_array.shape}")

print("\nProcessamento concluído!")

In [ ]:
!ls -lR ./*.npy | wc -l

In [ ]:
import numpy as np
import glob
import re

npy_files = glob.glob("magn2024*.npy")

def extract_doy(filename):
    match = re.search(r'magn2024(\d{3})\.npy', filename)
    if match:
        return int(match.group(1))
    return 0

npy_files.sort(key=extract_doy)

print(f"Encontrados {len(npy_files)} arquivos NPY para processar.")

all_data = []

for i, file in enumerate(npy_files):
    doy = extract_doy(file)
    print(f"Processando {file} (DOY {doy}) - {i+1}/{len(npy_files)}")

    data = np.load(file)

    all_data.append(data)

    print(f"  Shape: {data.shape}")

combined_data = np.concatenate(all_data, axis=0)

print(f"\nDados combinados. Shape final: {combined_data.shape}")

output_file = "MAGGIA-2024.npy"
np.save(output_file, combined_data)

print(f"\nArquivo {output_file} salvo com sucesso!")
print(f"Tamanho do arquivo: {combined_data.nbytes / (1024**2):.2f} MB")

In [ ]:
import numpy as np
from datetime import datetime, timedelta

def doy_to_date(year, doy):

    return datetime(year, 1, 1) + timedelta(days=doy - 1)

doys = {

    2024: {
        3: [61, 62, 63, 64, 65, 66, 67, 68, 69, 70,
            71, 72, 73, 74, 75, 76, 77, 78, 79, 80,
            81, 86, 87, 88, 89, 91],
        6: [153, 154, 155, 156, 157, 158, 159, 160, 161, 162,
            163, 164, 165, 166, 167, 168, 169, 170, 171, 173,
            174, 175, 176, 177, 180, 181],
        9: [247, 248, 249, 250, 251, 252, 253, 254, 255, 256,
            257, 258, 261, 262, 263, 264, 265, 266, 267, 268,
            269, 270, 271, 272, 274],
        12: [336, 337, 338, 339, 340, 341, 342, 343, 344, 345,
             346, 347, 349, 350, 351, 352, 353, 354, 355,
             356, 357, 358, 359, 360, 361, 362, 363, 364, 365]
    }
}

doy249_times = [
    "0000", "0010", "0015", "0020", "0030", "0040", "0045", "0050",
    "0100", "0110", "0115", "0120", "0130", "0140", "0145", "0150",
    "0200", "0210", "0215", "0220", "0230", "0240", "0245", "0250",
    "0300", "0310", "0315", "0320", "0330", "0340", "0345", "0350",
    "0400", "0410", "0415", "0420", "0430", "0440", "0445", "0450",
    "0500", "0510", "0515", "0520", "0530", "0540", "0545", "0550",
    "0600", "0610", "0615", "0620", "0630", "0640", "0645", "0650",
    "0700", "0710", "0715", "0720", "0730", "0740", "0745", "0750",
    "0800", "0810", "0815", "0820", "0830", "0840", "0845", "0850",
    "0900", "0910", "0915", "0920", "0930", "0940", "0945", "0950",
    "1000", "1010", "1015", "1020", "1030", "1040", "1045", "1050",
    "1100", "1110", "1115", "1120", "1130", "1140", "1145", "1150",
    "1200", "1210", "1215", "1220", "1230", "1240", "1245", "1250",
    "1300", "1310", "1315", "1320", "1330", "1340", "1345", "1350",
    "1400", "1410", "1415", "1420", "1430", "1440", "1445", "1450",
    "1500", "1510", "1515", "1520", "1530", "1540", "1545", "1550",
    "1600", "1610", "1615", "1620", "1630", "1640", "1645", "1650",
    "1700", "1710", "1715", "1720", "1730", "1740", "1745", "1750",
    "1800", "1810", "1815", "1820", "1830", "1840", "1845", "1850",
    "1900", "1910", "1915", "1920", "1930", "1940", "1945", "1950",
    "2000", "2010", "2015", "2020", "2030", "2040", "2045", "2050",
    "2100", "2110", "2115", "2120", "2130", "2140", "2145", "2150",
    "2200", "2210", "2215", "2220", "2230", "2240", "2245", "2250",
    "2300", "2310", "2315", "2320", "2330", "2340", "2345", "2350"
]

doy250_times = [
    "0000", "0010", "0015", "0020", "0030", "0040", "0045", "0050",
    "0100", "0110", "0115", "0120", "0130", "0140", "0145", "0150",
    "0200", "0210", "0215", "0220", "0230", "0240", "0245", "0250",
    "0300", "0310", "0315", "0320", "0330", "0340", "0345", "0350",
    "0400", "0410", "0415", "0420", "0430", "0440", "0445", "0450",
    "0500", "0510", "0515", "0520", "0530", "0540", "0545", "0550",
    "0600", "0610", "0615", "0620", "0630", "0640", "0645", "0650",
    "0700", "0710", "0715", "0720", "0730", "0740", "0745", "0750",
    "0800", "0810", "0815", "0820", "0830", "0840", "0845", "0850",
    "0900", "0910", "0915", "0920", "0930", "0940", "0945", "0950",
    "1000", "1010", "1015", "1020", "1030", "1040", "1045", "1050",
    "1100", "1110", "1115", "1120", "1130", "1140", "1145", "1150",
    "1200", "1210", "1220", "1230", "1240", "1250", "1300", "1310",
    "1320", "1330", "1340", "1350", "1400", "1410", "1420", "1430",
    "1440", "1450", "1500", "1510", "1520", "1530", "1540", "1550",
    "1600", "1610", "1620", "1630", "1640", "1650", "1700", "1710",
    "1720", "1730", "1740", "1750", "1800", "1810", "1820", "1830",
    "1840", "1850", "1900", "1910", "1920", "1930", "1940", "1950",
    "2000", "2010", "2020", "2030", "2040", "2050", "2100", "2110",
    "2120", "2130", "2140", "2150", "2200", "2210", "2220", "2230",
    "2240", "2250", "2300", "2310", "2320", "2330", "2340", "2350"
]

datetime_list = []

for year in doys.keys():
    for month in doys[year].keys():
        for doy in doys[year][month]:
            base_date = doy_to_date(year, doy)

            if year == 2024 and doy == 249:
                for time_str in doy249_times:
                    hours = int(time_str[:2])
                    minutes = int(time_str[2:])
                    dt = base_date + timedelta(hours=hours, minutes=minutes)
                    datetime_list.append(dt)

            elif year == 2024 and doy == 250:
                for time_str in doy250_times:
                    hours = int(time_str[:2])
                    minutes = int(time_str[2:])
                    dt = base_date + timedelta(hours=hours, minutes=minutes)
                    datetime_list.append(dt)

            elif year == 2024 and doy >= 251:
                for minute in range(0, 24 * 60, 10):
                    dt = base_date + timedelta(minutes=minute)
                    datetime_list.append(dt)

            else:
                for minute in range(0, 24 * 60, 15):
                    dt = base_date + timedelta(minutes=minute)
                    datetime_list.append(dt)

maggia_datetime_2022_2024 = np.array(datetime_list, dtype='datetime64[s]')

print(f"Total number of datetime points: {len(maggia_datetime_2022_2024)}")
print(f"First datetime: {maggia_datetime_2022_2024[0]}")
print(f"Last datetime: {maggia_datetime_2022_2024[-1]}")

np.save('maggia_datetime_2024.npy', maggia_datetime_2022_2024)

print("\nSample of first 10 datetime entries:")
for dt in maggia_datetime_2022_2024[:10]:
    print(dt)

print("\nSample of last 10 datetime entries:")
for dt in maggia_datetime_2022_2024[-10:]:
    print(dt)

In [ ]:
!ls -lh *.npy

In [ ]:
%pwd

In [ ]:
!ls *.npy

In [ ]:
%pwd

In [ ]:
maggia_2024 = np.load('MAGGIA-2024.npy')
print(np.shape(maggia_2024))
print(type(maggia_2024))
print(type(maggia_2024[0]))
print(type(maggia_2024[0][0]))
print(type(maggia_2024[0][0][0]))

In [ ]:
for i in range(len(maggia_2024)):
    print(np.shape(maggia_2024[i]))

In [ ]:
np.shape(maggia_2024)

In [ ]:
has_nan = np.any(np.isnan(maggia_2024))
print(has_nan)

count_nan = np.sum(np.isnan(maggia_2024))
print(count_nan)

In [ ]:
maggia_2024.shape

In [ ]:
np.set_printoptions(threshold=np.inf)

In [ ]:
print(maggia_2024[0][0])

In [ ]:
print(maggia_2024[0][-1])

In [ ]:
maggia_2024_shaped = np.reshape(maggia_2024, (-1, 241, 221))

In [ ]:
np.shape(maggia_2024_shaped)

In [ ]:
np.shape(maggia_2024_shaped.tolist())

# DATES ADJUSTS

## CREATE DATAFRAME

In [ ]:
maggia_dates = np.load('maggia_datetime_2024.npy')

In [ ]:
maggia_dates

In [ ]:
len(maggia_dates)

In [ ]:
import pandas as pd

data = {
    'DATETIME': maggia_dates,
    'TECMAP': maggia_2024_shaped.tolist()
}

df_mapas_maggia_2024 = pd.DataFrame(data)

df_mapas_maggia_2024

In [ ]:
df_mapas_maggia_2024['DATETIME'] = pd.to_datetime(df_mapas_maggia_2024['DATETIME'])

df_mapas_maggia_2024.set_index('DATETIME', inplace=True)

df_mapas_maggia_2024 = df_mapas_maggia_2024[(df_mapas_maggia_2024.index.minute == 0) | (df_mapas_maggia_2024.index.minute == 30)]

df3 = df_mapas_maggia_2024[
    ((df_mapas_maggia_2024.index.year == 2024) & (df_mapas_maggia_2024.index.month.isin([3, 6, 9, 12])))
]

print("DataFrame 3:")
print(df3)

# TF3 ADJUSTS

## 20:00 TO 04:00 UTC - 30-MINUTE TEMPORAL RESOLUTION

In [ ]:
np.array(df3.iloc[0]['TECMAP'])

In [ ]:
np.array(df3.iloc[0]['TECMAP']).shape

In [ ]:
df3_2000_0400 = pd.concat([
    df3.between_time('20:00', '20:00'),
    df3.between_time('20:30', '20:30'),
    df3.between_time('21:00', '21:00'),
    df3.between_time('21:30', '21:30'),
    df3.between_time('22:00', '22:00'),
    df3.between_time('22:30', '22:30'),
    df3.between_time('23:00', '23:00'),
    df3.between_time('23:30', '23:30'),
    df3.between_time('00:00', '00:00'),
    df3.between_time('00:30', '00:30'),
    df3.between_time('01:00', '01:00'),
    df3.between_time('01:30', '01:30'),
    df3.between_time('02:00', '02:00'),
    df3.between_time('02:30', '02:30'),
    df3.between_time('03:00', '03:00'),
    df3.between_time('03:30', '03:30'),
    df3.between_time('04:00', '04:00')
])

df3_2000_0400.sort_index(inplace=True)

print("\nDados das 20:00, 22:00, 00:00, 02:00 e 04:00:")
print(df3_2000_0400)

In [ ]:
df3.to_pickle("./df3_mapas_maggia_2024_30m.pkl")

In [ ]:
np.shape(np.array(df3.iloc[:]['TECMAP']))

In [ ]:
mapas3 = np.array(df3.iloc[:]['TECMAP'])

In [ ]:
np.shape(df3)

In [ ]:
np_mapas3 = []
for i in range(len(mapas3)):
    np_mapas3.append(mapas3[i])
np_mapas3 = np.array(np_mapas3)

In [ ]:
np.shape(np_mapas3)

In [ ]:
type(np_mapas3)

In [ ]:
np.save('mapas3_maggia_2024_30m.npy', np_mapas3)

In [ ]:
df3_2000_0400.to_pickle("./df3_mapas_maggia_2024_2000_0400_30m.pkl")

In [ ]:
mapas3_2000_0400 = np.array(df3_2000_0400.iloc[:]['TECMAP'])

In [ ]:
np.shape(mapas3_2000_0400)

In [ ]:
np_mapas3_2000_0400 = []
for i in range(len(mapas3_2000_0400)):
    np_mapas3_2000_0400.append(mapas3_2000_0400[i])
np_mapas3_2000_0400 = np.array(np_mapas3_2000_0400)

In [ ]:
np.shape(np_mapas3_2000_0400)

In [ ]:
type(np_mapas3_2000_0400)

In [ ]:
np.save('mapas3_maggia_2024_2000_0400_30m.npy', np_mapas3_2000_0400)

# DOWNLOAD OUTPUTS

In [ ]:
from google.colab import files

In [ ]:
%pwd

In [ ]:
!ls df*

In [ ]:
!ls maggia*

In [ ]:
# files.download('./maggia_2022_2024.npy')

In [ ]:
!ls -lh maggia*

In [ ]:
!ls -lh mapas*.npy

In [ ]:
# files.download('./maggia_2022_2024.npy')

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# %cp maggia_2022_2024.npy /content/drive/MyDrive/maggia_2022_2024.npy

In [ ]:
# %pwd

In [ ]:
# %cd /content/drive/MyDrive

In [ ]:
# !df -H /content/drive/MyDrive

In [ ]:
# files.download('./maggia_datetime_2022_2024.npy')

In [ ]:
# files.download('./df_mapas_maggia_2022_2024.pkl')

In [ ]:
# files.download('./df_mapas_maggia_2022_2024_0800.pkl')

In [ ]:
# files.download('./df_mapas_maggia_2022_2024_1600.pkl')

In [ ]:
# files.download('./df_mapas_maggia_2022_2024_2000_2200_0000_0200_0400.pkl')

In [ ]:
files.download('./df1_mapas_maggia_2022_2024.pkl')

In [ ]:
files.download('./df1_mapas_maggia_2022_2024_0800.pkl')

In [ ]:
files.download('./df1_mapas_maggia_2022_2024_1600.pkl')

In [ ]:
files.download('./df1_mapas_maggia_2022_2024_2000_2200_0000_0200_0400.pkl')

In [ ]:
files.download('./df2_mapas_maggia_2022_2024.pkl')

In [ ]:
files.download('./df2_mapas_maggia_2022_2024_0800.pkl')

In [ ]:
files.download('./df2_mapas_maggia_2022_2024_1600.pkl')

In [ ]:
files.download('./df2_mapas_maggia_2022_2024_2000_2200_0000_0200_0400.pkl')

In [ ]:
files.download('./mapas1_maggia_2022_2024.npy')

In [ ]:
files.download('./mapas1_maggia_2022_2024_0800.npy')

In [ ]:
files.download('./mapas1_maggia_2022_2024_1600.npy')

In [ ]:
files.download('./mapas1_maggia_2022_2024_2000_2200_0000_0200_0400.npy')

In [ ]:
files.download('./mapas2_maggia_2022_2024.npy')

In [ ]:
files.download('./mapas2_maggia_2022_2024_0800.npy')

In [ ]:
files.download('./mapas2_maggia_2022_2024_1600.npy')

In [ ]:
files.download('./mapas2_maggia_2022_2024_2000_2200_0000_0200_0400.npy')

In [ ]:
# files.download('./mapas_maggia_2022_2024_0800.npy')

In [ ]:
# files.download('./mapas_maggia_2022_2024_1600.npy')

In [ ]:
# files.download('./mapas_maggia_2022_2024_2000_2200_0000_0200_0400.npy')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls -lh df3*

In [ ]:
%cp df3_mapas_maggia_2024_30m.pkl /content/drive/MyDrive/df3_mapas_maggia_2024_30m.pkl

In [ ]:
%cp df3_mapas_maggia_2024_2000_0400_30m.pkl /content/drive/MyDrive/df3_mapas_maggia_2024_2000_0400_30m.pkl

In [ ]:
!ls -lh mapas3*

In [ ]:
%cp mapas3_maggia_2024_30m.npy /content/drive/MyDrive/mapas3_maggia_2024_30m.npy

In [ ]:
%cp mapas3_maggia_2024_2000_0400_30m.npy /content/drive/MyDrive/mapas3_maggia_2024_2000_0400_30m.npy